In [ ]:
!pip install -U sentence-transformers
!pip install datasets
!pip install transformers==4.21.3 
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from datasets import load_dataset
answersumm = load_dataset("alexfabbri/answersumm")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
from summarizer import Summarizer
body = """
The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.
"""
extractive_summary_model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# PreProcessing

In [ ]:
MAX_COUNT = 3000

In [ ]:
# Extract the questions
questions = []
for i in range(0,MAX_COUNT):
  question = answersumm["test"][i]["question"]["title"].lower()
  questions.append(question)

labels = []
answers = []
for i in range(0,MAX_COUNT):
  answer_list = []
  label_list = []
  for answer in answersumm["test"][i]["answers"]:
    answer_sentences = []
    label_sentences = []
    for sentence in answer['sents']:
      answer_sentences.append(sentence["text"].lower())
      label_sentences.append(sentence["label"][0])
    answer_list.append(answer_sentences)
    label_list.append(label_sentences)
  answers.append(answer_list)
  labels.append(label_list)
len(answers[0])

5

In [ ]:
input = []
output_labels = []
all_summaries = []
for ind,answer_val in enumerate(answers):
  final_summary = " "
  max_score = -1
  for val,answer in enumerate(answer_val):
    final_sentence = "<sep> "
    for sentence in answer:
      final_sentence = final_sentence +  sentence + " <sep> "
    score = int(answersumm["test"][ind]["answers"][val]["answer_details"]["score"])
    
    max_score = max(score,max_score)

    final_summary = final_summary + extractive_summary_model(final_sentence, ratio=0.6) + " "
  all_summaries.append(final_summary)

In [ ]:
all_summaries[0]

' <sep> if your date of volunteering was given as nov-2016 then i would assume that you volunteered for the whole month. <sep> that would greatly reduce your chances of being hired. <sep> if you were hired, and part of the reason for taking you and not someone else was the volunteering, then your job would be in extreme danger if this was found out. <sep> and seriously, if i read "volunteering for one day" i would have questions about your state of mind. <sep> specifying that you volunteered for one event on one day that has no real connection to your job does not really add much to your resume. <sep> even worse, as mentioned in the other answers, lying about the duration or making it unclear would be a very bad move. <sep> can i list one day of volunteer work as “month-year”? <sep> what you\'re suggesting isn\'t just simplifying your resume or cherry-picking what experience to list. <sep> unless specified, listing anything on your resume as "month - year" will imply that you spent the

In [ ]:
for summary in all_summaries:
  sentence_list = summary.split("<sep>")
  sentence_list = sentence_list[1:]
  print(sentence_list)

[' if your date of volunteering was given as nov-2016 then i would assume that you volunteered for the whole month. ', ' that would greatly reduce your chances of being hired. ', ' if you were hired, and part of the reason for taking you and not someone else was the volunteering, then your job would be in extreme danger if this was found out. ', ' and seriously, if i read "volunteering for one day" i would have questions about your state of mind. ', ' specifying that you volunteered for one event on one day that has no real connection to your job does not really add much to your resume. ', ' even worse, as mentioned in the other answers, lying about the duration or making it unclear would be a very bad move. ', ' can i list one day of volunteer work as “month-year”? ', " what you're suggesting isn't just simplifying your resume or cherry-picking what experience to list. ", ' unless specified, listing anything on your resume as "month - year" will imply that you spent the majority of th

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 6.76568598e-02  6.34958968e-02  4.87131290e-02  7.93049708e-02
   3.74480523e-02  2.65277759e-03  3.93749215e-02 -7.09844474e-03
   5.93614057e-02  3.15370187e-02  6.00980632e-02 -5.29051535e-02
   4.06067707e-02 -2.59308610e-02  2.98428200e-02  1.12691021e-03
   7.35149011e-02 -5.03819771e-02 -1.22386612e-01  2.37028301e-02
   2.97264811e-02  4.24768850e-02  2.56337877e-02  1.99519261e-03
  -5.69190904e-02 -2.71598231e-02 -3.29035483e-02  6.60248622e-02
   1.19007140e-01 -4.58791256e-02 -7.26214424e-02 -3.25840041e-02
   5.23413233e-02  4.50552776e-02  8.25301651e-03  3.67024057e-02
  -1.39415236e-02  6.53918907e-02 -2.64272541e-02  2.06390207e-04
  -1.36643294e-02 -3.62809822e-02 -1.95043571e-02 -2.89738551e-02
   3.94270420e-02 -8.84091035e-02  2.62427866e-03  1.36714270e-02
   4.83063459e-02 -3.11565790e-02 -1.17329150e-01 -5.11690192e-02
  -8.85287821e-02 -2.18962505e-02  1.42986560e-02  4.44168225e-02
  -1.34815350e-02  7.43392780e-02  2.66382433e-02 -1.98762640e-02
   1.79191

In [ ]:
from sentence_transformers import SentenceTransformer, util

"""
query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

#Load the model
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].tolist()

#Output passages & scores
for ind,score in enumerate(scores):
    print(score, docs[ind])
"""

'\nquery = "How many people live in London?"\ndocs = ["Around 9 Million people live in London", "London is known for its financial district"]\n\n#Load the model\nmodel = SentenceTransformer(\'sentence-transformers/multi-qa-mpnet-base-dot-v1\')\n\n#Encode query and documents\nquery_emb = model.encode(query)\ndoc_emb = model.encode(docs)\n\n#Compute dot score between query and all document embeddings\nscores = util.dot_score(query_emb, doc_emb)[0].tolist()\n\n#Output passages & scores\nfor ind,score in enumerate(scores):\n    print(score, docs[ind])\n'

In [ ]:
summary_encodings = []
for summary in all_summaries:
  sentence_list = summary.split("<sep>")
  sentence_list = sentence_list[1:]

  sent_encodings = []
  for sent in sentence_list:
    sent_encodings.append(model.encode(sent))
  summary_encodings.append(sent_encodings)

In [ ]:
len(summary_encodings[9])

17

In [ ]:
import numpy as np

In [ ]:
"""
doc_emb = model.encode(docs)
scores = util.dot_score(doc_emb[0]/np.linalg.norm(doc_emb[0]), doc_emb[1]/np.linalg.norm(doc_emb[1]))[0]
scores
"""

'\ndoc_emb = model.encode(docs)\nscores = util.dot_score(doc_emb[0]/np.linalg.norm(doc_emb[0]), doc_emb[1]/np.linalg.norm(doc_emb[1]))[0]\nscores\n'

In [ ]:
from sklearn.cluster import AgglomerativeClustering


X = np.array([[1, 2], [1, 4], [1, 0],
              [4, 2], [4, 4], [4, 0]])
clustering = AgglomerativeClustering(n_clusters=None,distance_threshold=0.65,affinity='cosine',linkage="complete").fit(X)
clustering

clustering.labels_


array([1, 1, 0, 0, 0, 0])

In [ ]:
from collections import Counter
cluster_summaries = []
for index,sent_encodings in enumerate(summary_encodings):
  clustering = AgglomerativeClustering(n_clusters=None,distance_threshold=0.65,affinity='cosine',linkage="complete").fit(sent_encodings)
  #print(clustering.labels_)
  clusters = clustering.labels_
  counts = Counter(clusters)
  intital_counts = Counter(clusters)
  num = 1
  while counts[num] != 0:
    counts[num] = counts[num] + counts[num - 1]
    num += 1
  summary_len = int(1 * counts[num - 1])
  cluster_count = summary_len * 0.5 / num
  #print(cluster_count)
  
  
  cluster_summary = ""
  #print(num)
  taken = Counter()

  for i in range(0,num):
    if intital_counts[clusters[i]] > cluster_count:
      taken[i] = 1

  final_sum = ""
  sentence_list = all_summaries[index].split("<sep>")
  sentence_list = sentence_list[1:]
  for i,sentence in enumerate(sentence_list):
    if taken[i] == 1:
      final_sum = final_sum + sentence + " "
  cluster_summaries.append(final_sum)
  

In [ ]:
cluster_summaries[0]

' if your date of volunteering was given as nov-2016 then i would assume that you volunteered for the whole month.   that would greatly reduce your chances of being hired.   if you were hired, and part of the reason for taking you and not someone else was the volunteering, then your job would be in extreme danger if this was found out.   and seriously, if i read "volunteering for one day" i would have questions about your state of mind.   specifying that you volunteered for one event on one day that has no real connection to your job does not really add much to your resume.   even worse, as mentioned in the other answers, lying about the duration or making it unclear would be a very bad move.   can i list one day of volunteer work as “month-year”?  '

In [ ]:
"""
from transformers import pipeline
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base")
"""

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
from transformers import BartForConditionalGeneration, AutoTokenizer

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq


In [ ]:
model = torch.load("pytorch_model.bin")
model_ckpt = "pytorch_model.bin"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
summarizer = BartForConditionalGeneration.from_pretrained(model_ckpt)
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=summarizer)

training_args = TrainingArguments(output_dir='bart-multi-news', num_train_epochs=1, warmup_steps=500, per_device_train_batch_size=1, per_device_eval_batch_size=1, weight_decay=0.01, logging_steps=10, push_to_hub=False, 
evaluation_strategy='steps', eval_steps=500, save_steps=1e6, 
gradient_accumulation_steps=16)

trainer = Trainer(model=summarizer, args=training_args, tokenizer=tokenizer, data_collator=seq2seq_data_collator, train_dataset=answersumm["train"], eval_dataset=answersumm["test"])
trainer.train()

In [ ]:
final_summaries = []
i = 0
for summary in cluster_summaries:
  final_summaries.append(summarizer(summary, truncation=True, max_length=150, min_length=50, do_sample=False)[0]['summary_text'])
  i = i +1
  print(i)

In [ ]:
final_summaries[0]

'lz granderson: if your date of volunteering was nov-2016, that would greatly reduce your chances of being hired . he says if you were hired, part of the reason for taking you and not someone else was the volunteering . granderson says lying about the duration or making it unclear would be a very bad move .'

In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge
rouge = Rouge()

In [ ]:
# Calculate Rouge Scores
avg_r1 = 0
avg_r2 = 0
avg_rl = 0
for i in range(MAX_COUNT):
  scores = rouge.get_scores(final_summaries[i],answersumm["test"]["summaries"][i][0][0] + " " +  answersumm["test"]["summaries"][i][0][1] )
  avg_r1 += scores[0]["rouge-1"]['f']
  avg_r2 += scores[0]["rouge-2"]['f']
  avg_rl += scores[0]["rouge-l"]['f']
           

avg_r1 = avg_r1 * 100.0 / MAX_COUNT
avg_r2 = avg_r2 * 100.0 / MAX_COUNT
avg_rl = avg_rl * 100.0 / MAX_COUNT
avg_r1,avg_r2,avg_rl

(25.29008591703774, 5.889565730601001, 21.10766150429837)

In [ ]:
answersumm["train"]["summaries"][0][0][0] + " " +  answersumm["train"]["summaries"][0][0][1]

"You might be better off not mentioning the volunteering on your CV at all, but if you do then you don't need to mention the dates. It is generally agreed that you cannot list a single day of volunteering with a simple month and year. However, it is probably not necessary to give dates for volunteer activities in any case."

In [ ]:
for i in range(MAX_COUNT):
    print("Question - " + answersumm["test"]["question"][i]["question"])
    print("Precited Summary - " + final_summaries[i])
    print("Final Summary - " + answersumm["test"]["summaries"][i][0][0] + " " +  answersumm["train"]["summaries"][i][0][1])
    print("-------")
    print()

Question - I am looking for volunteering experience that I can highlight on my resume or LinkedIn for an IT help-desk job, but mostly I find ones for a specific day, such as "Volunteers required for Garden Cleaning on this day".  Is it a good idea to mention the date of that one day volunteering for a whole month like "Nov-2016", although it was actually only one day?
Precited Summary - lz granderson: if your date of volunteering was nov-2016, that would greatly reduce your chances of being hired . he says if you were hired, part of the reason for taking you and not someone else was the volunteering . granderson says lying about the duration or making it unclear would be a very bad move .
Final Summary - You might be better off not mentioning the volunteering on your CV at all, but if you do then you don't need to mention the dates. It is generally agreed that you cannot list a single day of volunteering with a simple month and year. However, it is probably not necessary to give dates 

In [ ]:
answersumm["train"]["answers"][0][1]["answer_details"]["score"]

16

In [ ]:
answersumm["train"][0]["answers"]

[{'sents': [{'text': 'If your date of volunteering was given as Nov-2016 then I would assume that you volunteered for the whole month.',
    'label': [0],
    'label_summ': [0],
    'cluster_id': [[-1]]},
   {'text': "If I found out that you actually volunteered for one day only, I would assume that you were intentionally misleading me, that you were totally dishonest, and that you couldn't be trusted.",
    'label': [0],
    'label_summ': [0],
    'cluster_id': [[-1]]},
   {'text': 'That would greatly reduce your chances of being hired.',
    'label': [0],
    'label_summ': [0],
    'cluster_id': [[-1]]},
   {'text': 'If you were hired, and part of the reason for taking you and not someone else was the volunteering, then your job would be in extreme danger if this was found out.',
    'label': [0],
    'label_summ': [0],
    'cluster_id': [[-1]]},
   {'text': "You'd be better off not mentioning the volunteering at all.",
    'label': [1],
    'label_summ': [1],
    'cluster_id': [[0]]

In [ ]:
answers[0][0]

['interact professionally based on professional roles.',
 'say "mr. doe" or any standard professional way of referring to him that is appropriate.',
 'personal ties outside of work can be a challenge in the workplace for those involved.',
 'a parent-child relationship can be particularly difficult.',
 'in some cases, the child may struggle to step out of their parent\'s shadow and be known for their own work, rather than as "the boss\'s kid".',
 'in such cases, usually the individuals involved are trying to keep the personal and the professional separate.',
 'the most helpful thing for you to do is respect the same divide',
 ': base your interactions with them on their roles in the company, not on personal factors outside of work.',
 'if you say "your father requested this", you might be subtly contributing to a difficult situation.',
 'note: this may change if you develop more of a relationship with the people involved and interact more casually.',
 'but it would be the starting point